In [ ]:
import os
import time
import csv
import signal
import random
import itertools
import numpy as np
from concurrent.futures import ProcessPoolExecutor, TimeoutError

from qto.problems.facility_location_problem import generate_flp
from qto.problems.graph_coloring_problem import generate_gcp
from qto.problems.k_partition_problem import generate_kpp
from qto.problems.job_scheduling_problem import generate_jsp
from qto.problems.traveling_salesman_problem import generate_tsp
from qto.problems.set_cover_problem import generate_scp
from qto.solvers.optimizers import CobylaOptimizer, AdamOptimizer
from qto.solvers.qiskit import (
    PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver, 
    AerGpuProvider, AerProvider, FakeBrisbaneProvider, FakeKyivProvider, FakeTorinoProvider, DdsimProvider,
)

import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列


设置执行规模

In [2]:
num_cases = 10  # 每个benchmark测试的case数量
problem_scale = 1 # 问题规模，2表示测试前2个规模，无GPU运行basline时较慢

定义问题

In [3]:
flp_problems_pkg, flp_configs_pkg = generate_flp(num_cases, [(1, 2), (2, 3), (3, 3), (3, 4)][:problem_scale], 1, 20)
gcp_problems_pkg, gcp_configs_pkg = generate_gcp(num_cases, [(3, 1), (3, 2), (4, 1), (4, 2)][:problem_scale])
kpp_problems_pkg, kpp_configs_pkg = generate_kpp(num_cases, [(4, 2, 3), (5, 3, 4), (6, 3, 5), (7, 3, 6)][:problem_scale], 1, 20)
jsp_problems_pkg, jsp_configs_pkg = generate_jsp(num_cases, [(2, 2, 3), (2, 3, 4), (3, 3, 5), (3, 4, 6)][:problem_scale], 1, 20)
scp_problems_pkg, scp_configs_pkg = generate_scp(num_cases, [(4, 4), (5, 5), (6, 6), (7, 7)][:problem_scale])

configs_pkg = flp_configs_pkg + gcp_configs_pkg + kpp_configs_pkg + jsp_configs_pkg + scp_configs_pkg
with open(f"1_table.config", "w") as file:
    for pkid, configs in enumerate(configs_pkg):
        for problem in configs:
            file.write(f'{pkid}: {problem}\n')


如果成功配置GPU版本  
请替换：  
provider = cpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,  
为  
provider = gpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,

In [4]:
new_path = '1_table_depth'

problems_pkg = flp_problems_pkg + gcp_problems_pkg + kpp_problems_pkg + jsp_problems_pkg + scp_problems_pkg

metrics_lst = ['culled_depth', 'num_params']
solvers = [PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver]
headers = ["pkid", 'method', 'layers'] + metrics_lst

def process_layer(prb, num_layers, solver, metrics_lst):
    opt = CobylaOptimizer(max_iter=200)
    ddsim = DdsimProvider()
    cpu = AerProvider()
    gpu = AerGpuProvider()
    used_solver = solver(
        prb_model = prb,
        optimizer = opt,
        # 根据配置的环境选择CPU或GPU
        provider = cpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,
        # provider = gpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,
        num_layers = num_layers,
        shots = 1024,
    )
    metrics = used_solver.circuit_analyze(metrics_lst)
    return metrics

if __name__ == '__main__':
    set_timeout = 60 * 60 * 24 # Set timeout duration
    num_complete = 0
    with open(f'{new_path}.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # Write headers once

    num_processes_cpu = os.cpu_count() // 2
    with ProcessPoolExecutor(max_workers=num_processes_cpu) as executor:
        futures = []
        for solver in solvers:
            for pkid, problems in enumerate(problems_pkg):
                for problem in problems:
                    if solver == ChocoSolver:
                        num_layers = 1
                    else:
                        num_layers = 5
                    future = executor.submit(process_layer, problem, num_layers, solver, metrics_lst)
                    futures.append((future, pkid, solver.__name__, num_layers))

        start_time = time.perf_counter()
        for future, pkid, solver, num_layers in futures:
            current_time = time.perf_counter()
            remaining_time = max(set_timeout - (current_time - start_time), 0)
            diff = []
            try:
                result = future.result(timeout=remaining_time)
                diff.extend(result)
                print(f"Task for problem {pkid}, num_layers {num_layers} executed successfully.")
            except MemoryError:
                diff.append('memory_error')
                print(f"Task for problem {pkid}, num_layers {num_layers} encountered a MemoryError.")
            except TimeoutError:
                diff.append('timeout')
                print(f"Task for problem {pkid}, num_layers {num_layers} timed out.")
            finally:
                row = [pkid, solver, num_layers] + diff
                with open(f'{new_path}.csv', mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow(row)  # Write row immediately
                num_complete += 1
                if num_complete == len(futures):
                    print(f'Data has been written to {new_path}.csv')
                    for process in executor._processes.values():
                        os.kill(process.pid, signal.SIGTERM)

Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license -

In [5]:
file_path = '1_table_depth.csv'

df = pd.read_csv(file_path)

grouped_df = df.groupby(['pkid', 'layers', 'method'], as_index=False).agg({
    "culled_depth": 'mean',
})

values = ["culled_depth"]
pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=values)

method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([values, method_order]))

pivot_df


culled_depth                                   
     PenaltySolver CyclicSolver HeaSolver ChocoSolver
pkid                                                 
0             40.0         65.0      30.0        44.0
1            135.2        200.0      60.0       167.3
2             82.2        142.0      40.0       114.9
3             52.0        115.0      35.0        96.4
4            131.8        131.8      45.0        82.4

In [6]:
new_path = '1_table_evaluate'

problems_pkg = list(
    itertools.chain(
        enumerate(flp_problems_pkg),
        enumerate(gcp_problems_pkg),
        enumerate(kpp_problems_pkg),
        enumerate(jsp_problems_pkg),
        enumerate(scp_problems_pkg),
    )
)

solvers = [PenaltySolver, CyclicSolver, HeaSolver, ChocoSolver]
evaluation_metrics = ['best_solution_probs', 'in_constraints_probs', 'ARG', 'iteration_count', 'classcial', 'quantum', 'run_times']
headers = ['pkid', 'pbid', 'layers', "variables", 'constraints', 'method'] + evaluation_metrics

def process_layer(prb, num_layers, solver):
    opt = CobylaOptimizer(max_iter=200)
    ddsim = DdsimProvider()
    cpu = AerProvider()
    gpu = AerGpuProvider()
    prb.set_penalty_lambda(400)
    used_solver = solver(
        prb_model = prb,
        optimizer = opt,
        # 根据配置的环境选择CPU或GPU
        # provider = cpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,
        provider = gpu if solver in [PenaltySolver, CyclicSolver, HeaSolver] else ddsim,
        num_layers = num_layers,
        shots = 1024,
    )
    used_solver.solve()
    eval = used_solver.evaluation()
    time = list(used_solver.time_analyze())
    run_times = used_solver.run_counts()
    return eval + time + [run_times]

if __name__ == '__main__':
    all_start_time = time.perf_counter()
    set_timeout = 60 * 60 * 2 # Set timeout duration
    num_complete = 0

    with open(f'{new_path}.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # 写入标题

    num_processes_cpu = os.cpu_count()
    # pkid-pbid: 问题包序-包内序号
    for pkid, (diff_level, problems) in enumerate(problems_pkg):
        for solver in solvers:
            if solver in [PenaltySolver, CyclicSolver, HeaSolver]:
                num_processes = 2**(4 - diff_level) + 1
            else:
                num_processes = 100

            with ProcessPoolExecutor(max_workers=num_processes) as executor:
                futures = []
                if solver == ChocoSolver:
                    layer = 1
                else:
                    layer = 5

                for pbid, prb in enumerate(problems):
                    print(f'{pkid}-{pbid}, {layer}, {solver} build')
                    future = executor.submit(process_layer, prb, layer, solver)
                    futures.append((future, prb, pkid, pbid, layer, solver.__name__))

                start_time = time.perf_counter()
                for future, prb, pkid, pbid, layer, solver in futures:
                    current_time = time.perf_counter()
                    remaining_time = max(set_timeout - (current_time - start_time), 0)
                    diff = []
                    try:
                        metrics = future.result(timeout=remaining_time)
                        diff.extend(metrics)
                        print(f"Task for problem {pkid}-{pbid} L={layer} {solver} executed successfully.")
                    except MemoryError:
                        print(f"Task for problem {pkid}-{pbid} L={layer} {solver} encountered a MemoryError.")
                        for dict_term in evaluation_metrics:
                            diff.append('memory_error')
                    except TimeoutError:
                        print(f"Task for problem {pkid}-{pbid} L={layer} {solver} timed out.")
                        for dict_term in evaluation_metrics:
                            diff.append('timeout')
                    except Exception as e:
                        print(f"An error occurred: {e}")
                    finally:
                        row = [pkid, pbid, layer, len(prb.variables), len(prb.lin_constr_mtx), solver] + diff
                        with open(f'{new_path}.csv', mode='a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(row)  # Write row immediately
                        num_complete += 1
                        if num_complete == len(futures):
                            print(f'problem_pkg_{pkid} has finished')
                            for process in executor._processes.values():
                                os.kill(process.pid, signal.SIGTERM)
    print(f'Data has been written to {new_path}.csv')
    print(time.perf_counter()- all_start_time)

0-0, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-1, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-2, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-3, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-4, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-5, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-6, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-7, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-8, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
0-9, 5, <class 'qto.solvers.qiskit.penalty.PenaltySolver'> build
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24
Restricted license - for non-production use only - expires 2025-11-24


In [8]:
df = pd.read_csv('1_table_evaluate.csv')

df = df.drop(columns=['pbid'])
df = df[df['ARG'] <= 100000]

grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean',
    'classcial':'mean',
    'run_times':'mean',
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid', 'variables', 'constraints'], columns='method', values=["best_solution_probs",'in_constraints_probs', 'ARG'])
method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([["best_solution_probs",'in_constraints_probs', 'ARG'], method_order]))

pivot_df

best_solution_probs                         \
                                 PenaltySolver CyclicSolver HeaSolver   
pkid variables constraints                                              
0    6         3                      3.125000     5.546875  7.910156   
1    12        6                      0.468750    20.097656  0.087891   
2    8         6                      2.951389    12.196181  1.703559   
3    7         4                      7.480469     8.652344  2.158203   
4    9         4                      2.841797     4.277344  1.347656   

                                       in_constraints_probs               \
                           ChocoSolver        PenaltySolver CyclicSolver   
pkid variables constraints                                                 
0    6         3             60.546875            24.042969    44.013672   
1    12        6             36.474609             1.787109    55.527344   
2    8         6             80.815972             5.371094    33.333333   
3    7         4             70.751953            13.505859    42.636719   
4    9         4             43.203125             8.183594    14.335938   

                                                            ARG               \
                            HeaSolver ChocoSolver PenaltySolver CyclicSolver   
pkid variables constraints                                                     
0    6         3            23.642578  100.000000     43.546872    22.709950   
1    12        6             0.332031  100.000000    156.114337    32.856728   
2    8         6             5.219184  100.000000    350.236933   120.617531   
3    7         4             6.943359  100.000000     77.991690    45.444239   
4    9         4             4.707031   99.941406    747.679488   572.898532   

                                                    
                             HeaSolver ChocoSolver  
pkid variables constraints                          
0    6         3             43.646085    0.107228  
1    12        6            186.677975    0.316197  
2    8         6            341.880640    0.144874  
3    7         4            106.112591    0.042610  
4    9         4            804.849165    0.499561